In [ ]:
# here the goal is to evaluate the results from notebook 2 

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set(style="whitegrid")

train_pred = pd.read_parquet("../results/baseline/pred_train.parquet")
val_pred   = pd.read_parquet("../results/baseline/pred_val.parquet")

print("Train predictions:", len(train_pred))
print("Val predictions:",   len(val_pred))

train_pred.head()
# loading the deepface prediction files generated in notebook 2 and cheking the number of rows to confirm everything is fine

Train predictions: 7000
Val predictions: 2100


,pred_gender,pred_gender_score,pred_race,pred_race_score,error,file,race_true,gender_true
0,Woman,88.418669,black,43.714210,None,train/60423.jpg,Black,Female
1,Man,97.433734,black,78.286773,None,train/45029.jpg,Black,Female
2,Woman,99.483669,latino hispanic,41.124514,None,train/81730.jpg,Black,Female
3,Man,89.559507,indian,58.377320,None,train/72069.jpg,Black,Female
4,Man,64.608073,black,34.814405,None,train/37655.jpg,Black,Female


In [36]:
train_gt = pd.read_csv("../data/processed/balanced_train.csv")
val_gt   = pd.read_csv("../data/processed/balanced_val.csv")

train = train_pred.merge(train_gt, on="file", how="left")
val   = val_pred.merge(val_gt,   on="file", how="left")

train = train.rename(columns={"gender": "gender_true", "race": "race_true"})
val   = val.rename(columns={"gender": "gender_true", "race": "race_true"})

print(len(train), len(val))
train.head()
# merging predictions with their true labels so I can measure accuracy nd bias

7000 2100


,pred_gender,pred_gender_score,pred_race,pred_race_score,error,file,race_true,gender_true,age,gender_true,race_true,service_test
0,Woman,88.418669,black,43.714210,None,train/60423.jpg,Black,Female,20-29,Female,Black,True
1,Man,97.433734,black,78.286773,None,train/45029.jpg,Black,Female,30-39,Female,Black,False
2,Woman,99.483669,latino hispanic,41.124514,None,train/81730.jpg,Black,Female,20-29,Female,Black,True
3,Man,89.559507,indian,58.377320,None,train/72069.jpg,Black,Female,40-49,Female,Black,False
4,Man,64.608073,black,34.814405,None,train/37655.jpg,Black,Female,40-49,Female,Black,False


In [37]:
gender_map = {
    "Man":   "Male",
    "Woman": "Female",
}

race_map = {
    "black":           "Black",
    "white":           "White",
    "asian":           "East Asian",     
    "indian":          "Indian",
    "latino hispanic": "Latino_Hispanic",
    "middle eastern":  "Middle Eastern",
    "others":          "Other",
}

train["pred_gender"] = train["pred_gender"].replace(gender_map)
val["pred_gender"]   = val["pred_gender"].replace(gender_map)

train["pred_race"] = train["pred_race"].replace(race_map)
val["pred_race"]   = val["pred_race"].replace(race_map)

print("unique predicted genders:", train["pred_gender"].unique())
print("unique predicted races:",   train["pred_race"].unique())
# map deepface label names to the label format used in my dataset so i can compare matching categories instead of mismatched strings

unique predicted genders: ['Female' 'Male']
unique predicted races: ['Black' 'Latino_Hispanic' 'Indian' 'White' 'East Asian' 'Middle Eastern']


In [38]:
print(train.columns)

Index(['pred_gender', 'pred_gender_score', 'pred_race', 'pred_race_score',
       'error', 'file', 'race_true', 'gender_true', 'age', 'gender_true',
       'race_true', 'service_test'],
      dtype='object')


In [39]:
train = train.loc[:, ~train.columns.duplicated()]
val   = val.loc[:, ~val.columns.duplicated()]

print(train.columns)
# remove duplicate columns accidentally created during the merge

Index(['pred_gender', 'pred_gender_score', 'pred_race', 'pred_race_score',
       'error', 'file', 'race_true', 'gender_true', 'age', 'service_test'],
      dtype='object')


In [40]:
train = add_error_cols(train)
val   = add_error_cols(val)

train.head()
# creating binary columns: 1 = correct, 0 = wrong -> these make accuracy calculations easie

,pred_gender,pred_gender_score,pred_race,pred_race_score,error,file,race_true,gender_true,age,service_test,gender_ok,race_ok
0,Female,88.418669,Black,43.714210,None,train/60423.jpg,Black,Female,20-29,True,1,1
1,Male,97.433734,Black,78.286773,None,train/45029.jpg,Black,Female,30-39,False,0,1
2,Female,99.483669,Latino_Hispanic,41.124514,None,train/81730.jpg,Black,Female,20-29,True,1,0
3,Male,89.559507,Indian,58.377320,None,train/72069.jpg,Black,Female,40-49,False,0,0
4,Male,64.608073,Black,34.814405,None,train/37655.jpg,Black,Female,40-49,False,0,1
